In [56]:
#!/usr/bin/env python
# coding: utf-8

#import sys
from gensim.models.doc2vec import Doc2Vec
import numpy as np
from ckiptagger import WS
import pandas as pd
import re
import random
from datetime import datetime
import os
import logging
from LinkedIn import linkedin_crawler
import getpass
import pymysql
import pandas as pd
import time
start_time = time.time()

def recommendation():
    #模型讀取
    model = Doc2Vec.load("Doc2Vec_0826_v500_m1_e50_w5.model")
   
    #CKIP斷詞資料
    #os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
    #ws = WS('data', disable_cuda=False)
    ws = WS('data') #有使用GPU / Tensorflow-GPU再註解掉這行，啟用上面兩行
    
    
    #print("--- %s seconds ---" % (time.time() - start_time))
    # updated 8/25 ==
    def query_mysql():
        db = pymysql.connect(host='mysql', user='user',password='user',
                               database='project',charset="utf8",port=3306)
        cursor = db.cursor()

        maxm = pd.read_sql("select max(USER) as 'maxm' from Login_2",db)
        q=maxm.loc[0,'maxm']
        sql_2 = "select * from Login_2 where USER={}".format(q) 

        data = pd.read_sql(sql_2, db) #利用pandas直接获取数据
        data = data.loc[0,'PSW']
        db.close()
        return data
    
    def insert_recom2(a_list):   
        db = pymysql.connect(host='mysql', user='user',password='user',
                               database='project',charset="utf8",port=3306)
        cursor = db.cursor()
        
        # colect 30
        
        code_list = [i[0] for i in a_list]
        sql_2 = "select distinct Code, companyName, jobName, salary, C, longitude, latitude from s104_full where Code in ({})".format(code_list).replace('[','').replace(']','')
        data = pd.read_sql(sql_2, db) #利用pandas直接获取数据
        data.insert(7,'Similarity',0)
        data_code_list = data['Code'].tolist()
        data = data.set_index('Code')

        for info in a_list:
            if info[0] in data_code_list:
                data.loc[info[0],'Similarity'] = info[1]
        data = data.reset_index()
        df = data
        #print(df)
        #print("--- %s seconds ---" % (time.time() - start_time))
        # insert 30
        for i in range(0,30):
            if i < df.shape[0]:
                C1=df.loc[i]['Code']
                C2=df.loc[i]['companyName']
                C3=df.loc[i]['jobName']
                C4=df.loc[i]['salary'] 
                C5=df.loc[i]['C'] 
                C6=df.loc[i]['longitude'] 
                C7=df.loc[i]['latitude']
                C8=df.loc[i]['Similarity']
            else:
                C1=""
                C2=""
                C3=""
                C4=""
                C5=""
                C6=0
                C7=0
                C8=0
                
            sql_1 = "insert into recom_2(Code, companyName, jobName, salary, C, longitude, latitude, Similarity) values('%s','%s','%s','%s','%s', %s ,%s ,%s)" % \
                    (C1,C2,C3,C4,C5,C6,C7,C8)
            
            try:
                cursor.execute(sql_1) # 执行sql语句
                db.commit() # 提交到数据库执行
            except:
                db.rollback() # 如果发生错误则回滚
        db.close()
        #print("--- %s seconds ---" % (time.time() - start_time))
    # 8/25 end ==
    
    #消除符號
    def regularize_(target):
        #return re.sub("[0-9【】◎★/◆※⦿\%\[\]\-\"\“\”\(\)（）{}\'=●▲▼《》○]", "", target).replace('\r',' ').replace('\n',' ')
        ##Vincent改 - 斷詞結構改一下
        return re.sub("[【】◎★/◆※⦿\%\[\]\-\"\“\”\(\)（）{}\'=●▲▼.〈〉《》○?？!，。！，,:　：~.、；_／ ]", "★", target).replace('\r','★').replace('\n','★')
    #定義斷詞功能_這邊是改版給單句用的
    def word_seg(sentence_list): #輸入包含多斷文章(句子)的list
        sepped = ws([sentence_list],
                    sentence_segmentation=True,
                    #segment_delimiter_set={'?', '？', '!', '！', '。', ',','，', ';', ':', '、','：','\\n','\\r','.',' ','_'})
                    ##Vincent改 - 斷詞結構改一下2
                    segment_delimiter_set={'\\n','\\r','.',' ','_','★'})
        return sepped[0]

    #比較兩個斷詞組的相似性
    def compare_similarity(p1, p2):
        d2v_similarity = np.dot(model.infer_vector(p1), model.infer_vector(p2)) / (np.linalg.norm(model.infer_vector(p1))*np.linalg.norm(model.infer_vector(p2)))
        return d2v_similarity
        

    doc_origin = query_mysql() # == amend 08/25

    #目標履歷去雜質、斷字斷詞
    doc = regularize_(doc_origin)
    doc = word_seg(doc)

    #輸出目標履歷資料
    print('履歷內容： %s' % (doc_origin))
    print('--------------------------------------------')

    #套用模型到目標履歷詞組
    inferred_vector = model.infer_vector(doc)
    #取得以相似度排列的職缺list
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    
    #輸出推薦 # == amend 08/25
    a_list=[]
    for index in range(0,30):
        #a_list.append(sims[index][0])
        a_list.append(sims[index])
    print('------------in mySQLdb------------------',a_list)
    insert_recom2(a_list)
    
    print("--- %s seconds ---" % (time.time() - start_time))

if __name__ == '__main__':
    
    recommendation()
    
    


2020-08-27 07:52:19,034 : INFO : loading Doc2Vec object from Doc2Vec_0826_full.model
2020-08-27 07:52:19,930 : INFO : loading vocabulary recursively from Doc2Vec_0826_full.model.vocabulary.* with mmap=None
2020-08-27 07:52:19,930 : INFO : loading trainables recursively from Doc2Vec_0826_full.model.trainables.* with mmap=None
2020-08-27 07:52:19,931 : INFO : loading wv recursively from Doc2Vec_0826_full.model.wv.* with mmap=None
2020-08-27 07:52:19,932 : INFO : loading docvecs recursively from Doc2Vec_0826_full.model.docvecs.* with mmap=None
2020-08-27 07:52:19,933 : INFO : loaded Doc2Vec_0826_full.model
2020-08-27 07:52:22,502 : INFO : precomputing L2-norms of doc weight vectors


履歷內容： HEY JUDE
--------------------------------------------
------------in mySQLdb------------------ [('6vny3', 0.5669575929641724), ('6bzyb', 0.5254725813865662), ('6xr2s', 0.5025080442428589), ('37p13', 0.4814065396785736), ('6ubmh', 0.47761401534080505), ('6pdvk', 0.47629818320274353), ('6xuco', 0.47555434703826904), ('3i8jq', 0.464337557554245), ('69ozq', 0.462085485458374), ('6swyb', 0.4581959545612335), ('4rjy0', 0.442859947681427), ('6ypzk', 0.437854528427124), ('6hq18', 0.43362998962402344), ('6j26i', 0.4335951507091522), ('7086j', 0.43309450149536133), ('3676v', 0.43116623163223267), ('6wxkr', 0.42892053723335266), ('6xb66', 0.42798951268196106), ('3snup', 0.42777568101882935), ('6yu4z', 0.4276735186576843), ('64t4g', 0.4246920943260193), ('6ryx1', 0.4225388169288635), ('34izi', 0.42165762186050415), ('6wpvd', 0.42110419273376465), ('6kucn', 0.42062628269195557), ('6gdhu', 0.42021819949150085), ('6yu4w', 0.4189473092556), ('6x74h', 0.4183298349380493), ('625xz', 0.418250083923